In [ ]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import math

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

from Area2_analysis.funcs import angle_between, process_train_test
from Area2_analysis.funcs import get_sses_pred, get_sses_mean, nans, fit_and_predict
from Area2_analysis.funcs import sub_and_predict

# Single Lag

In [ ]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Han_20171207"
filename = foldername + monkey + "_COactpas_TD.nwb"

dataset_5ms = NWBDataset(filename, split_heldout=False)

speed = np.sqrt(np.sum(dataset_5ms.data['hand_vel'][:].T**2,axis=0)).to_numpy().reshape((-1,1))
dataset_5ms.add_continuous_data(speed,'speed')

acceleration = np.diff(speed, axis = 0, prepend=[speed[0]])
dataset_5ms.add_continuous_data(acceleration,'acceleration')

xy_vel = dataset_5ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_5ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

dataset_5ms.resample(5)
dataset_5ms.smooth_spk(40, name='smth_40')
# dataset_5ms.smooth_spk(20, name='smth_20')
bin_width = dataset_5ms.bin_width
print(bin_width)

In [ ]:
n_dims = 20 # for PCA

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset_5ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_5ms.data.spikes_smth_40)
print(all_data.shape)

if not np.isnan(all_data).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(all_data)
    pca = PCA(n_components=n_dims,random_state = 42)
    PCA_data = pca.fit_transform(X)
else:
    scaler = StandardScaler()
    X = scaler.fit_transform(all_data[~np.isnan(all_data).any(axis=1)])
    pca = PCA(n_components=n_dims,random_state = 42)
    X = pca.fit(X)
    PCA_data = nans([all_data.shape[0],n_dims])
    idx = 0
    for dp in all_data:
        dp = dp.reshape((1, -1))
        if np.isnan(dp).any():
            dp_pca = nans([1,n_dims])
        else:
            dp_pca = pca.transform(scaler.transform(dp))
        PCA_data[idx,:] = dp_pca
        idx+=1
print(PCA_data.shape)
dataset_5ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

#make dictionary for trial condition (reaching directions) for Stratified CV
active_trials_idx = np.array(dataset_5ms.trial_info.loc[trial_mask]['trial_id'])
cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_5ms.trial_info['cond_dir'] == direction) & (dataset_5ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_5ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

In [ ]:
# Prepare for PCA plotting
pred_range = (-100, 500)
trial_mask = active_mask
n_timepoints = int((pred_range[1] - pred_range[0])/dataset_5ms.bin_width)
n_trials = dataset_5ms.trial_info.loc[trial_mask].shape[0]
active_data = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=pred_range, ignored_trials=~trial_mask)
active_trials_pca = nans([n_trials,n_timepoints,n_dims])
i = 0
for idx, trial in active_data.groupby('trial_id'):
    active_trials_pca[i,:,:]=trial.PCA.to_numpy()
    i+=1
print(active_trials_pca.shape)

plot_dir = np.array([0,45,90,135,180,225,270,315]) # limit plot directions to reduce cluttering
directions = np.array([0,45,90,135,180,225,270,315])
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

# define some useful time points
move_idx=0
# ret_idx = 200

import matplotlib as mpl
cmap = plt.get_cmap('coolwarm',len(plot_dir))
custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(len(plot_dir))]

plot_dims = 10

fig,ax=plt.subplots(plot_dims,1,figsize=(10,20))
for i in range(plot_dims):
    for j in range(len(plot_dir)):
        color = custom_palette[j]
        dir_idx = np.argwhere(directions == plot_dir[j])[0]
        cond_mean_proj = np.mean(active_trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,i] 
        pca_mean = np.mean(active_data.PCA.to_numpy(),axis = 0)[i]
        ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
        ax[i].axvline(move_idx, color='k',linewidth = .5)
        # ax[i].axvline(ret_idx, color='k',linewidth = .5)
#         ax[i].set_xlim([0,T])
        ax[i].set_ylim([-10, 10])
        ax[i].axhline(0,color ='k',ls = '--')
        if i<plot_dims-1:
            ax[i].set_xticks([])
        else:
            ax[i].set_xlabel('Time (ms)')
            
        ax[i].set_yticks([])
        ax[i].set_ylabel('Dim. '+str(i+1))

    ax[0].set_title('PCA Projections')
    
plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')

### Timing plots

In [ ]:
n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')

figDir = "/Users/sherryan/area2_population_analysis/figures/timing/"

In [ ]:
plot_range = (-40, 100)
x_axis = np.arange(plot_range[0], plot_range[1], dataset_5ms.bin_width)
passive_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)
active_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)

plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis, trial.force['y'], color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.force['y'], color='red', linewidth=0.5)
plt.xlabel('Time after bump time (ms)')
plt.ylabel('Force to manipulandum (N)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Force aligned to move_onset')
plt.tight_layout()
plt.savefig(figDir + monkey + '_force.png', dpi = 'figure')
plt.close()

In [ ]:
plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.speed, color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.speed, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Speed aligned to move_onset')
plt.tight_layout()
plt.savefig(figDir + monkey + '_speed.png', dpi = 'figure')
plt.close()

In [ ]:
plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.acceleration, color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.acceleration, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand acceleration (cm/s^2)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Acceleration aligned to move_onset')
plt.tight_layout()
plt.savefig(figDir + monkey + '_acc.png', dpi = 'figure')
plt.close()

In [ ]:
active_trials_spikes = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_spikes.append(np.sum(trial.spikes,axis=1))
passive_trials_spikes = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_spikes.append(np.sum(trial.spikes,axis=1))
plt.plot(x_axis,np.sum(active_trials_spikes,axis = 0)/dataset_5ms.bin_width*1000/len(active_trials_spikes)/n_neurons,"-o",color = 'k',label = 'Active')
plt.plot(x_axis,np.sum(passive_trials_spikes,axis = 0)/dataset_5ms.bin_width*1000/len(passive_trials_spikes)/n_neurons,"-o",color = 'red',label = 'Passive')

plt.legend()
plt.title('Peristimulus aligned to move_onset')
plt.ylabel('Trial-average, neuron-average, FR (per sec)')
plt.xlabel('Time (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    
plt.tight_layout()
plt.savefig(figDir + monkey + '_psth.png', dpi = 'figure')
plt.close()

In [ ]:
plot_range = (-500, 600)
x_axis = np.arange(plot_range[0], plot_range[1], dataset_5ms.bin_width)
active_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)
passive_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)

plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.speed, color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.speed, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Speed aligned to move_onset')
plt.axvline(120, color = 'k',linestyle = '--')
plt.tight_layout()
plt.savefig(figDir + monkey + '_speed_whole.png', dpi = 'figure')
plt.close()

In [ ]:
plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial['hand_acc']['x'], color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial['hand_acc']['x'], color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand acceleration (cm/s^2)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Acceleration_x aligned to move_onset')
plt.axvline(120, color = 'k',linestyle = '--')
plt.tight_layout()
plt.savefig(figDir + monkey + '_acc_x_whole.png', dpi = 'figure')
plt.close()

plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial['hand_acc']['y'], color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial['hand_acc']['y'], color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand acceleration (cm/s^2)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Acceleration_y aligned to move_onset')
plt.axvline(120, color = 'k',linestyle = '--')
plt.tight_layout()
plt.savefig(figDir + monkey + '_acc_y_whole.png', dpi = 'figure')
plt.close()

In [ ]:
active_trials_spikes = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_spikes.append(np.sum(trial.spikes,axis=1))
passive_trials_spikes = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_spikes.append(np.sum(trial.spikes,axis=1))
plt.plot(x_axis,np.sum(active_trials_spikes,axis = 0)/dataset_5ms.bin_width*1000/len(active_trials_spikes)/n_neurons,"-o",color = 'k',label = 'Active')
plt.plot(x_axis,np.sum(passive_trials_spikes,axis = 0)/dataset_5ms.bin_width*1000/len(passive_trials_spikes)/n_neurons,"-o",color = 'red',label = 'Passive')

plt.legend()
plt.title('Peristimulus aligned to move_onset')
plt.ylabel('Trial-average, neuron-average, FR (per sec)')
plt.xlabel('Time (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    
plt.tight_layout()
plt.savefig(figDir + monkey + '_psth_whole.png', dpi = 'figure')
plt.close()

## with Neurons

In [ ]:
lag_axis = np.arange(-300,300,20)
x_field = 'spikes_smth_40'
y_field ='hand_acc'
trial_mask = active_mask

# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
plot_dim = 'x' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']

figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"

dim = n_neurons

In [ ]:
pred_range = (-100, 500)
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)
curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field, cond_dict = cond_dict)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting ' + y_field + ' ' + str(pred_range))
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
eff_weights = curr_coef_array[idx_max,:,:]

#subtract predictions with primary decoding dimensions (at time with max R2)
sub_coef_array = nans([len(lag_axis),2,dim])
sub_r2_array = nans([len(lag_axis)])

for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,eff_weights, cond_dict = cond_dict)
    sub_r2_array[i] = r2
    sub_coef_array[i,:,:] = coef

plt.plot(lag_axis,sub_r2_array)
plt.title('R2 score projecting out #1 t_max dim')
idx_max = np.argmax(sub_r2_array)
time_max = lag_axis[idx_max]
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.show()
plt.close()

aff_weights = sub_coef_array[idx_max,:,:]

In [ ]:
_, eff_weights, _ = fit_and_predict(dataset_5ms, active_mask, 'move_onset_time', (-100, 500), -40, x_field, 'hand_acc', cond_dict = cond_dict)
_, aff_weights,_ = sub_and_predict(dataset_5ms, active_mask, 'move_onset_time', (-100,500), 80,x_field,'hand_acc',eff_weights, cond_dict = cond_dict)

In [ ]:
eff_weights_X = eff_weights[0,:]
print(eff_weights_X.shape)
aff_weights_X = aff_weights[0,:]
print(aff_weights_X.shape)
plt.figure(figsize = (15,6))
plt.bar(np.arange(n_neurons)-0.2,abs(eff_weights_X),.4,label = 'Eff_X')
plt.bar(np.arange(n_neurons)+0.2,abs(aff_weights_X),.4,label = 'Aff_X')
plt.title('Neuron Weights')
plt.legend()

In [ ]:
Ki_x = []
for i in range(n_neurons):
    Wa = aff_weights_X[i]
    We = eff_weights_X[i]
    if abs(Wa) > 0.01 or abs(We) > 0.01:
        # Ki_x.append((abs(Wa) - abs(We)) / max(abs(Wa), abs(We)))
        if abs(Wa) > abs(We):
            Ki_x.append(abs(Wa - We) / max(abs(Wa), abs(We)))
        else:
            Ki_x.append(-abs(Wa - We) / max(abs(Wa), abs(We)))
print(len(Ki_x))
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
fig.suptitle('Relative weighting to X_acc between Afference and Efference')
axs[0].violinplot(Ki_x,showmeans = True)
axs[1].boxplot(Ki_x)
axs[0].set_ylabel('Weighting')
for ax in axs:
    ax.yaxis.grid(True)
    ax.set_xticks([])
    ax.set_xlabel('All neurons')

In [ ]:
plt.figure(figsize = (8,6),dpi=300)
plt.bar(np.arange(len(Ki_x)),sorted(Ki_x))
plt.ylabel('Weighting')
plt.xlabel('Neuron #')

In [ ]:
ranges = [(-100,500),(-100,120),(380,500)]
labels = ['_whole_acc_','_long_acc_','_late_acc_']

for pred_range, label in zip(ranges, labels):
    x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)
    curr_r2_array = nans([len(lag_axis)])
    curr_coef_array = nans([len(lag_axis),2,dim])
    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field, cond_dict = cond_dict)
        curr_r2_array[i] = r2
        curr_coef_array[i,:,:] = coef

    idx_max = np.argmax(curr_r2_array)
    time_max = lag_axis[idx_max]
    _, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field, cond_dict = cond_dict)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
    plt.close()

    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
    plt.close()

    plt.plot(lag_axis, curr_r2_array)
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.title('R2 score predicting ' + y_field + ' ' + str(pred_range))
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
    plt.close()

    # ang_dist_to_max = nans([len(lag_axis)])
    # for i in range(0, len(curr_coef_array)):
    #     ang_dist_to_max[i] = math.degrees(angle_between(curr_coef_array[i,0,:],curr_coef_array[idx_max,0,:]))
    # plt.scatter(lag_axis, ang_dist_to_max)
    # plt.title('Angular distance to X-vel decoding dim at t_max')
    # plt.xlabel('Time lag (ms)')
    # plt.ylabel('Angular distance (degrees)')
    # plt.show()

    weights = curr_coef_array[idx_max,:,:]
    for iter in range(0,3):  
        #subtract predictions with primary decoding dimensions (at time with max R2)
        sub_coef_array = nans([len(lag_axis),2,dim])
        sub_r2_array = nans([len(lag_axis)])

        for i in range(len(lag_axis)):
            lag = lag_axis[i]
            r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights, cond_dict = cond_dict)
            sub_r2_array[i] = r2
            sub_coef_array[i,:,:] = coef

        plt.plot(lag_axis,sub_r2_array)
        plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
        idx_max = np.argmax(sub_r2_array)
        time_max = lag_axis[idx_max]
        plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
        plt.legend()
        plt.xlabel('Time lag (ms)')
        plt.ylabel('R2')
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
        plt.close()

        _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights, cond_dict = cond_dict)
        for trial_dir, color in zip(plot_dir, colors):
            cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
            for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
                plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        plt.xlabel('Time (ms)')
        plt.ylabel(plot_dim + '_' + y_field)
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
        plt.close()

        #stack the decoding dimensions to be projected out
        weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

## with PCA

In [ ]:
x_field = 'PCA'
y_field ='hand_acc'
lag_axis = np.arange(-300,300,20)
trial_mask = active_mask
# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
plot_dim = 'x' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']

figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"

dim = n_dims

In [ ]:
ranges = [(-100,500),(-100,120),(380,500)]
labels = ['_whole_acc_','_long_acc_','_late_acc_']

for pred_range, label in zip(ranges, labels):
    x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)
    curr_r2_array = nans([len(lag_axis)])
    curr_coef_array = nans([len(lag_axis),2,dim])
    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field, cond_dict = cond_dict)
        curr_r2_array[i] = r2
        curr_coef_array[i,:,:] = coef

    idx_max = np.argmax(curr_r2_array)
    time_max = lag_axis[idx_max]
    _, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field, cond_dict = cond_dict)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
    plt.close()

    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
    plt.close()

    plt.plot(lag_axis, curr_r2_array)
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.title('R2 score predicting ' + y_field + ' ' + str(pred_range))
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
    plt.close()

    weights = curr_coef_array[idx_max,:,:]
    for iter in range(0,3):  
        #subtract predictions with primary decoding dimensions (at time with max R2)
        sub_coef_array = nans([len(lag_axis),2,dim])
        sub_r2_array = nans([len(lag_axis)])

        for i in range(len(lag_axis)):
            lag = lag_axis[i]
            r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights,cond_dict = cond_dict)
            sub_r2_array[i] = r2
            sub_coef_array[i,:,:] = coef

        plt.plot(lag_axis,sub_r2_array)
        plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
        idx_max = np.argmax(sub_r2_array)
        time_max = lag_axis[idx_max]
        plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
        plt.legend()
        plt.xlabel('Time lag (ms)')
        plt.ylabel('R2')
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
        plt.close()

        _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights, cond_dict = cond_dict)
        for trial_dir, color in zip(plot_dir, colors):
            cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
            for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
                plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        plt.xlabel('Time (ms)')
        plt.ylabel(plot_dim + '_' + y_field)
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
        plt.close()

        #stack the decoding dimensions to be projected out
        weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

# Multi Lags

In [ ]:
dataset_50ms = NWBDataset(filename, split_heldout=False)
xy_vel = dataset_50ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_50ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

dataset_50ms.resample(50)
print(dataset_50ms.bin_width)

In [ ]:
n_dims = 20 # for PCA

active_mask = (~dataset_50ms.trial_info.ctr_hold_bump) & (dataset_50ms.trial_info.split != 'none')
passive_mask = (dataset_50ms.trial_info.ctr_hold_bump) & (dataset_50ms.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset_50ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_50ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_50ms.data.spikes)
print(all_data.shape)
if not np.isnan(all_data).any():
    scaler = StandardScaler()
    X = scaler.fit_transform(all_data)
    pca = PCA(n_components=n_dims,random_state = 42)
    PCA_data = pca.fit_transform(X)
else:
    scaler = StandardScaler()
    X = scaler.fit_transform(all_data[~np.isnan(all_data).any(axis=1)])
    pca = PCA(n_components=n_dims,random_state = 42)
    X = pca.fit(X)
    PCA_data = nans([all_data.shape[0],n_dims])
    idx = 0
    for dp in all_data:
        dp = dp.reshape((1, -1))
        if np.isnan(dp).any():
            dp_pca = nans([1,n_dims])
        else:
            dp_pca = pca.transform(scaler.transform(dp))
        PCA_data[idx,:] = dp_pca
        idx+=1

print(PCA_data.shape)
dataset_50ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

In [ ]:
active_data = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range=(-400, 700), ignored_trials=~trial_mask)
for idx, trial in active_data.groupby('trial_id'):
    n_timepoints = trial.shape[0]
    break
print(n_timepoints,'time bins')

active_trials_neuron = nans([n_trials,n_timepoints,n_neurons])
active_trials_vel = nans([n_trials,n_timepoints,2])
active_trials_acc = nans([n_trials,n_timepoints,2])
active_trials_pca = nans([n_trials,n_timepoints,n_dims])
i = 0
for idx, trial in active_data.groupby('trial_id'):
    active_trials_neuron[i,:,:]=trial.spikes.to_numpy()
    active_trials_vel[i,:,:]=trial.hand_vel.to_numpy()
    active_trials_acc[i,:,:]=trial.hand_acc.to_numpy()
    active_trials_pca[i,:,:]=trial.PCA.to_numpy()
    i+=1
print(active_trials_neuron.shape)
print(active_trials_vel.shape)
print(active_trials_acc.shape)
print(active_trials_pca.shape)

#make dictionary for trial condition (reaching directions) for Stratified CV
active_trials_idx = np.array(dataset_50ms.trial_info.loc[trial_mask]['trial_id'])
cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_50ms.trial_info['cond_dir'] == direction) & (dataset_50ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_50ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

### with Neurons

In [ ]:
data_range = [-400,700]
figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"
active_x = active_trials_neuron
active_y = active_trials_acc
y_type = 'acceleration'

ranges = [(-100,500),(0,120),(-100,120)]
labels = ['whole_acc','early_acc','long_acc']

dim = n_neurons

In [ ]:
for pred_range, label in zip(ranges, labels):

    idx1 = int((pred_range[0] - data_range[0])/dataset_50ms.bin_width)
    idx2 = int(n_timepoints - (data_range[1]-pred_range[1])/dataset_50ms.bin_width)
    
    if pred_range == (-100,500):
        t_before_range = range(0,201,50);
        t_after_range = range(0,201,50);        
    else:        
        t_before_range = range(0,301,50);
        t_after_range = range(0,501,50);

    multi_R2s = nans([len(t_before_range),len(t_after_range)])
    multi_coefs = []
    j,k=0,0
    for time_before in t_before_range:
        coef_arr = []
        for time_after in t_after_range:
            print('Predicting with',-time_before, 'to', time_after,'ms neural data')

            bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
            bins_current= 1 #Whether to use concurrent time bin of neural data
            bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

            n_total_bins = bins_before + bins_current + bins_after

            X =  nans([n_trials,idx2-idx1,n_total_bins*dim])
            i = 0
            for trial_data in active_x:
                trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
                trial_hist = trial_hist[idx1:idx2,:,:]
                trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
                X[i,:,:] = trial_hist_flat
                i+=1
            y = active_y[:,idx1:idx2,:]

            lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
            X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
            y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
            lr_all.fit(X_reshaped, y_reshaped)
            print(lr_all.best_params_['alpha'])

            skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
            true_concat = nans([(n_trials*(idx2-idx1)),2])
            pred_concat = nans([(n_trials*(idx2-idx1)),2])
            trial_save_idx = 0
            for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
                #split training and testing by trials
                X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
                lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
                lr.fit(X_train, y_train)
                y_test_predicted = lr.predict(X_test)
                n = y_test_predicted.shape[0]
                true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
                pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
                trial_save_idx += n

            sses =get_sses_pred(true_concat,pred_concat)
            sses_mean=get_sses_mean(true_concat)
            multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
            print('R2:',multi_R2s[j,k])
            coef_arr.append(lr_all.best_estimator_.coef_)
            k += 1
        j += 1
        k = 0
        multi_coefs.append(coef_arr)

    fig, ax = plt.subplots()
    im = ax.imshow(multi_R2s)
    ax.set_xlabel('Length of lagging info')
    ax.set_ylabel('Length of leading info')

    ax.set_xticks(np.arange(len(t_after_range)))
    ax.set_yticks(np.arange(len(t_before_range)))
    ax.set_xticklabels(labels=t_after_range)
    ax.set_yticklabels(labels=t_before_range)

    ax.set_title('R2 predicting ' + str(pred_range) + ' ' + y_type +'\nwith different lagging/leading info')
    fig.tight_layout()

    for i in range(len(t_before_range)):
        for j in range(len(t_after_range)):
            text = ax.text(j, i, str(int(multi_R2s[i, j]*1000)/1000),
                           ha="center", va="center", color="w")

    plt.tight_layout()
    plt.savefig(figDir + monkey + '_multi_' + label + '.png', dpi = 'figure')
    plt.close()
    
    if pred_range == (-100,500):
        coef_X = multi_coefs[-1][-1][0] #which entry's weights to use

        t_label = np.arange(-200,201,50)

        n_weights = len(t_before_range) + len(t_after_range) - 1
        coef_X_reshaped = coef_X.reshape(n_weights,n_neurons)
        angDist_array = nans([n_weights,n_weights])
        for i in range(n_weights):
            for j in range(n_weights):
                angDist_array[i,j] = math.degrees(angle_between(coef_X_reshaped[i,:],coef_X_reshaped[j,:]))
        fig, ax = plt.subplots(figsize=(6, 6))
        im = ax.imshow(angDist_array)
        ax.set_xlabel('Bin time (ms)')
        ax.set_ylabel('Bin time (ms)')

        ax.set_xticks(np.arange(len(t_label)))
        ax.set_yticks(np.arange(len(t_label)))
        ax.set_xticklabels(labels=t_label)
        ax.set_yticklabels(labels=t_label)

        ax.set_title("Angle between weight vectors at time points")
        fig.tight_layout()

        for i in range(len(t_label)):
            for j in range(len(t_label)):
                text = ax.text(j, i, str(int(angDist_array[i, j])),
                               ha="center", va="center", color="w")
        plt.tight_layout()
        plt.savefig(figDir + monkey + '_multi_' + label + '_deg.png', dpi = 'figure')
        plt.close()

### with PCA

In [ ]:
data_range = [-400,700]
figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"
active_x = active_trials_pca
active_y = active_trials_acc
y_type = 'acceleration'

ranges = [(-100,500),(0,120),(-100,120)]
labels = ['whole_acc','early_acc','long_acc']

dim = n_dims

In [ ]:
for pred_range, label in zip(ranges, labels):

    idx1 = int((pred_range[0] - data_range[0])/dataset_50ms.bin_width)
    idx2 = int(n_timepoints - (data_range[1]-pred_range[1])/dataset_50ms.bin_width)

    if pred_range == (-100,500):
        t_before_range = range(0,201,50);
        t_after_range = range(0,201,50);        
    else:        
        t_before_range = range(0,301,50);
        t_after_range = range(0,501,50);

    multi_R2s = nans([len(t_before_range),len(t_after_range)])
    multi_coefs = []
    j,k=0,0
    for time_before in t_before_range:
        coef_arr = []
        for time_after in t_after_range:
            print('Predicting with',-time_before, 'to', time_after,'ms neural data')

            bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
            bins_current= 1 #Whether to use concurrent time bin of neural data
            bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

            n_total_bins = bins_before + bins_current + bins_after

            X =  nans([n_trials,idx2-idx1,n_total_bins*dim])
            i = 0
            for trial_data in active_x:
                trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
                trial_hist = trial_hist[idx1:idx2,:,:]
                trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
                X[i,:,:] = trial_hist_flat
                i+=1
            y = active_y[:,idx1:idx2,:]

            lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
            X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
            y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
            lr_all.fit(X_reshaped, y_reshaped)
            print(lr_all.best_params_['alpha'])

            skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
            true_concat = nans([(n_trials*(idx2-idx1)),2])
            pred_concat = nans([(n_trials*(idx2-idx1)),2])
            trial_save_idx = 0
            for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
                #split training and testing by trials
                X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
                lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
                lr.fit(X_train, y_train)
                y_test_predicted = lr.predict(X_test)
                n = y_test_predicted.shape[0]
                true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
                pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
                trial_save_idx += n

            sses =get_sses_pred(true_concat,pred_concat)
            sses_mean=get_sses_mean(true_concat)
            multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
            print('R2:',multi_R2s[j,k])
            coef_arr.append(lr_all.best_estimator_.coef_)
            k += 1
        j += 1
        k = 0
        multi_coefs.append(coef_arr)

    fig, ax = plt.subplots()
    im = ax.imshow(multi_R2s)
    ax.set_xlabel('Length of lagging info')
    ax.set_ylabel('Length of leading info')

    ax.set_xticks(np.arange(len(t_after_range)))
    ax.set_yticks(np.arange(len(t_before_range)))
    ax.set_xticklabels(labels=t_after_range)
    ax.set_yticklabels(labels=t_before_range)

    ax.set_title('R2 predicting ' + str(pred_range) + ' ' + y_type +'\nwith different lagging/leading info')
    fig.tight_layout()

    for i in range(len(t_before_range)):
        for j in range(len(t_after_range)):
            text = ax.text(j, i, str(int(multi_R2s[i, j]*1000)/1000),
                           ha="center", va="center", color="w")

    plt.tight_layout()
    plt.savefig(figDir + monkey + '_multi_' + label + '.png', dpi = 'figure')
    plt.close()
    
    if pred_range == (-100,500):
        coef_X = multi_coefs[-1][-1][0] #which entry's weights to use

        t_label = np.arange(-200,201,50)

        n_weights = len(t_before_range) + len(t_after_range) - 1
        coef_X_reshaped = coef_X.reshape(n_weights,dim)
        angDist_array = nans([n_weights,n_weights])
        for i in range(n_weights):
            for j in range(n_weights):
                angDist_array[i,j] = math.degrees(angle_between(coef_X_reshaped[i,:],coef_X_reshaped[j,:]))
        fig, ax = plt.subplots(figsize=(6, 6))
        im = ax.imshow(angDist_array)
        ax.set_xlabel('Bin time (ms)')
        ax.set_ylabel('Bin time (ms)')

        ax.set_xticks(np.arange(len(t_label)))
        ax.set_yticks(np.arange(len(t_label)))
        ax.set_xticklabels(labels=t_label)
        ax.set_yticklabels(labels=t_label)

        ax.set_title("Angle between weight vectors at time points")
        fig.tight_layout()

        for i in range(len(t_label)):
            for j in range(len(t_label)):
                text = ax.text(j, i, str(int(angDist_array[i, j])),
                               ha="center", va="center", color="w")
        plt.tight_layout()
        plt.savefig(figDir + monkey + '_multi_' + label + '_deg.png', dpi = 'figure')
        plt.close()